In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [ ]:
# Data preproccess
strategy = tf.distribute.MirroredStrategy()
data = pd.read_csv("/content/drive/MyDrive/767project/Shakespeare_data.csv")
dataset = data['PlayerLine']
subset_size = int(len(dataset) * 0.10)
subset_indices = np.random.choice(range(len(dataset)), size=subset_size, replace=False)
subset_dataset = dataset[subset_indices]
dataset = subset_dataset
corpus = []
with strategy.scope():
    for line in dataset:
        lowercase_line = line.lower()
        corpus.append(lowercase_line)
corpus[:10]

['inventoried, and every particle and utensil',
 "many a time hath banish'd norfolk fought",
 'have burst their cerements, why the sepulchre,',
 'life, to make thee a double-dealer, which, out of',
 'o, do not swear, my lord of buckingham.',
 'i, now the voice of the recorded law,',
 'indeed my mother! or were you both our mothers,',
 'and of our athens, thine and ours, to take',
 'he would show most love. when yet he was but',
 'disguised like muscovites, in shapeless gear,']

In [ ]:
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
# convert into token
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2LMHeadModel.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token


inputs = tokenizer(corpus, max_length=40, padding='max_length', truncation=True, return_tensors='tf')
labels = inputs.input_ids
labels = tf.where(labels == tokenizer.pad_token_id, -100, labels)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
# train the model
batch_size = 8
for epoch in range(3):
    for i in range(0, len(inputs.input_ids), batch_size):
        batch_input_ids = inputs.input_ids[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]
        with tf.GradientTape() as tape:
            outputs = model(batch_input_ids, labels=batch_labels, training=True)
            loss = outputs.loss

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f"Epoch {epoch+1}, Loss: {loss.numpy()}")


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1, Loss: [4.930343]
Epoch 2, Loss: [4.178693]
Epoch 3, Loss: [3.5174983]


In [ ]:
def generate_text(prompt, max_length):
    # Load GPT-2
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = TFGPT2LMHeadModel.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    input_ids = tokenizer.encode(prompt, return_tensors='tf')

    # Call the generate function in GPT
    generated_text_samples = model.generate(
        input_ids,
        max_length=max_length + len(input_ids[0]),
        num_return_sequences=5,
        no_repeat_ngram_size=2,
        repetition_penalty=1.5,
        top_p=0.92,
        temperature=0.85,
        do_sample=True,
        top_k=0
    )

    # Decode the generated text
    generated_texts = []
    for i, beam in enumerate(generated_text_samples):
        decoded_text = tokenizer.decode(beam, skip_special_tokens=True)
        generated_texts.append(decoded_text)
        print(f"Generated Text {i+1}: {decoded_text}")

    return generated_texts

# Given the prompt and Length
prompt_text = "long live the king"
desired_length = 10
generated_texts = generate_text(prompt_text, desired_length)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: long live the king, as he laught them."
"Does
Generated Text 2: long live the king. He says, "The King of Queens is
Generated Text 3: long live the king's standard, he is now forced to make a
Generated Text 4: long live the king's life.
When it comes to tradition,
Generated Text 5: long live the king is mighty.
With those two forces he will


In [ ]:
prompt_text = "Dream to be true"
desired_length = 10
generated_texts = generate_text(prompt_text, desired_length)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: Dream to be true"? Why can't we have any kind of relationship
Generated Text 2: Dream to be true.
And after a pretty long night, I
Generated Text 3: Dream to be true" ©Getty Images
..
Generated Text 4: Dream to be true and break down barriers with clarity.
Sadd
Generated Text 5: Dream to be true, any time.
My name is Ryan Link


In [ ]:
prompt_text = "King of the king"
desired_length = 10
generated_texts = generate_text(prompt_text, desired_length)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: King of the king was born at that time. The princess Anna,
Generated Text 2: King of the king's court, and his son; he came to
Generated Text 3: King of the king.


Diplomacy - four elements from
Generated Text 4: King of the king, as King Steven and his siblings married in a
Generated Text 5: King of the king


Speak to him first and give his


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

# Save to Google Drive
model_path = "/content/drive/My Drive/gpt2_my_model"
model.save_pretrained(model_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import TFGPT2LMHeadModel

loaded_model = TFGPT2LMHeadModel.from_pretrained(model_path)


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/My Drive/gpt2_my_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
